In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

In [ ]:
def init_weights(shape):
    """Initialize the weights and create variables"""
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [ ]:
def init_bias(shape):
    """Initialize the bias and create variables"""
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [ ]:
def conv2d(x, W):
    """Create a 2D convolution layer"""
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [ ]:
def max_pool2by2(x):
    """Create a max pooling layer"""
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [ ]:
def convolutional_layer(x,shape):
    """Call all the functions and call relu activation function"""
    weight = init_weights(shape)
    bias = init_bias([shape[3]])
    return tf.nn.relu(conv2d(x, weight)+bias)

In [ ]:
def normal_layer(input_layer,size):
    """Create a fully connected layer"""
    input_size = int(input_layer.get_shape()[1])
    weight = init_weights([input_size,size])
    bias = init_bias([size])
    return tf.matmul(input_layer,weight) + bias

In [ ]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [ ]:
x_image = tf.reshape(x,[-1,28,28,1])

In [ ]:
convo_1 = convolutional_layer(x_image,shape=[3,3,1,32])
convo_1_pooling = max_pool2by2(convo_1)

In [ ]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[3,3,32,64])
convo_2_pooling = max_pool2by2(convo_2)

In [ ]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_layer(convo_2_flat,1024))

In [ ]:
#Dropout layer placeholder
drop_out_placeholder = tf.placeholder(tf.float32)

In [ ]:
full_one_dropout= tf.nn.dropout(full_layer_one,keep_prob=drop_out_placeholder)

In [ ]:
y_pred = normal_layer(full_one_dropout,10)

In [ ]:
#Cross entropy loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [ ]:
#optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
saver = tf.train.Saver()

In [ ]:
#Create a session and run the model

with tf.Session() as sess:
    sess.run(init)
    
    epoch = 1000
    for i in range(1,epoch):
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,drop_out_placeholder:0.2})
        
        if i%100 ==0:
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc, feed_dict = {x:mnist.test.images, y_true:mnist.test.labels,drop_out_placeholder:0.2}))
            print('\n')
    save_path = saver.save(sess, "./output/model.ckpt")
    print("Model saved in file: %s" % save_path)

In [ ]:
# Saving the complete session

with tf.Session() as sess:
    writer = tf.summary.FileWriter("output",sess.graph)
    print(sess.run(init))
    writer.close()